In [52]:
import pandas as pd
import numpy as np
import lightgbm as lgb
from sklearn.model_selection import TimeSeriesSplit
import sys
import ta
import yaml
import os
sys.path.insert(1, '/home/kenny/algotrading/backtesting')
from backtest import * 
from strategies import *
from config import *
from model import *
from dataset import *
from data import *
from sklearn.metrics import (
    log_loss,
    roc_auc_score,
    f1_score,
    mean_squared_error,
    accuracy_score,
    r2_score,
    mean_absolute_error,
)

In [53]:
path = '/home/kenny/algotrading/model_training/data_trained/data.csv'
df = pd.read_csv(path, index_col=False)
df.set_index(['Time'], inplace=True)
df.head()

,Open,High,Low,Close,Volume
Time,,,,,
2022-01-03 23:06:00,46293.89,46363.56,46293.89,46313.06,35.34215
2022-01-03 23:07:00,46313.05,46334.60,46262.16,46299.58,16.99444
2022-01-03 23:08:00,46299.58,46326.37,46294.44,46294.45,12.07571
2022-01-03 23:09:00,46294.44,46310.55,46250.33,46293.49,13.63073
2022-01-03 23:10:00,46293.48,46383.68,46290.00,46351.99,31.07135


In [54]:
data = Data(df)
data.create_features_and_target(task_type='classification')

In [55]:
import json

res_24 = '/home/kenny/algotrading/model_training/feature_exps_results/classification/LightGBM/res-24'
with open(res_24) as f:
    result = json.load(f)
features_24 = result['features']

In [56]:
model = Model(model_name='LightGBM')
result = model.train(data, metric='roc_auc',features=features_24, is_optimized=True)

# optimize_results = '/home/kenny/algotrading/model_training/optimization_results'
# model.optimize(data, optimize_results, features_24)

[LightGBM] [Warning] early_stopping_round is set=378, early_stopping_rounds=378 will be ignored. Current value: early_stopping_round=378
[LightGBM] [Warning] lambda_l1 is set=50, reg_alpha=0.0 will be ignored. Current value: lambda_l1=50
[LightGBM] [Warning] bagging_fraction is set=0.2, subsample=1.0 will be ignored. Current value: bagging_fraction=0.2
[LightGBM] [Warning] min_gain_to_split is set=12.192376416446765, min_split_gain=0.0 will be ignored. Current value: min_gain_to_split=12.192376416446765
[LightGBM] [Warning] lambda_l2 is set=90, reg_lambda=0.0 will be ignored. Current value: lambda_l2=90
[LightGBM] [Warning] feature_fraction is set=0.2, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.2
[LightGBM] [Warning] bagging_freq is set=1, subsample_freq=0 will be ignored. Current value: bagging_freq=1
[1]	training's binary_logloss: 0.693147
[2]	training's binary_logloss: 0.693147
[3]	training's binary_logloss: 0.693116
[4]	training's binary_logloss: 0.6927

In [57]:
model.model_trained

LGBMClassifier(bagging_fraction=0.2, bagging_freq=1, early_stopping_rounds=378,
               feature_fraction=0.2, lambda_l1=50, lambda_l2=90,
               learning_rate=0.23988865249336208, max_depth=5,
               min_gain_to_split=12.192376416446765, n_estimators=10000,
               num_leaves=1360)

#### Saving model to onnx format

In [71]:
import onnxmltools
import onnx
from onnxmltools.convert.common.data_types import FloatTensorType, Int64TensorType

def custom_shape_calculator(input_shapes):
    # Calculate the output shape based on the input shape
    output_shape = (input_shapes[0][0], input_shapes[1][1])
    return output_shape

# initial_types = [("float_input", FloatTensorType([None, len(features_24)]))]
# custom_shape_calculators=[custom_shape_calculator]
initial_types = [("input", FloatTensorType([None, len(features_24)]))]
onnx_model = onnxmltools.convert_lightgbm(model.model_trained,initial_types=initial_types, target_opset=9)
onnxmltools.utils.save_model(onnx_model, 'model.onnx')


In [69]:
test = data.df[:20][features_24]
test

,Open,High,Low,Volume,Close,SMA_3,SMA_5,SMA_15,SMA_30,SMA_50,...,close_1,close_2,close_3,close_4,close_5,high_1,high_2,high_3,high_4,high_5
Time,,,,,,,,,,,,,,,,,,,,,
2022-01-04 00:45:00,46276.98,46324.27,46257.91,20.01309,46311.71,46298.213333,46310.666,46370.960000,46344.401333,46378.6868,...,46276.97,46305.96,46324.28,46334.41,46391.94,46305.97,46324.28,46351.65,46410.42,46398.17
2022-01-04 00:46:00,46311.71,46334.87,46280.76,11.92522,46298.51,46295.730000,46303.486,46362.328667,46343.249000,46375.9980,...,46311.71,46276.97,46305.96,46324.28,46334.41,46324.27,46305.97,46324.28,46351.65,46410.42
2022-01-04 00:47:00,46298.50,46312.61,46291.95,5.98359,46304.50,46304.906667,46299.530,46355.416667,46343.678000,46373.6678,...,46298.51,46311.71,46276.97,46305.96,46324.28,46334.87,46324.27,46305.97,46324.28,46351.65
2022-01-04 00:48:00,46304.51,46324.00,46304.49,6.22721,46323.99,46309.000000,46303.136,46350.518667,46345.447333,46371.6740,...,46304.50,46298.51,46311.71,46276.97,46305.96,46312.61,46334.87,46324.27,46305.97,46324.28
2022-01-04 00:49:00,46324.00,46324.00,46290.00,15.71796,46290.00,46306.163333,46305.742,46340.122000,46345.714000,46368.5520,...,46323.99,46304.50,46298.51,46311.71,46276.97,46324.00,46312.61,46334.87,46324.27,46305.97
2022-01-04 00:50:00,46290.01,46296.53,46260.42,11.74081,46270.00,46294.663333,46297.400,46329.795333,46344.720333,46363.7380,...,46290.00,46323.99,46304.50,46298.51,46311.71,46324.00,46324.00,46312.61,46334.87,46324.27
2022-01-04 00:51:00,46265.44,46310.00,46261.60,15.76963,46309.99,46289.996667,46299.696,46324.315333,46346.208333,46360.3534,...,46270.00,46290.00,46323.99,46304.50,46298.51,46296.53,46324.00,46324.00,46312.61,46334.87
2022-01-04 00:52:00,46310.00,46310.00,46258.77,10.34325,46280.99,46286.993333,46294.994,46317.311333,46346.497000,46355.8458,...,46309.99,46270.00,46290.00,46323.99,46304.50,46310.00,46296.53,46324.00,46324.00,46312.61
2022-01-04 00:53:00,46281.00,46290.10,46240.56,16.15161,46240.57,46277.183333,46278.310,46307.677333,46346.338667,46351.9396,...,46280.99,46309.99,46270.00,46290.00,46323.99,46310.00,46310.00,46296.53,46324.00,46324.00


#### Inference model from onnx format

In [86]:
import onnxruntime as rt

session = rt.InferenceSession("model.onnx")

input_name = session.get_inputs()[0].name
output_name = session.get_outputs()[0].name

pred_onnx = session.run( [output_name], {input_name: test.values.astype(np.float32)})[0]
pred_onnx

2023-02-25 14:15:54.395957846 [W:onnxruntime:, execution_frame.cc:835 VerifyOutputSizes] Expected shape from model of {1} does not match actual shape of {20} for output label


array([1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 0, 1],
      dtype=int64)

In [82]:
if output_data[0].shape != (1,):
    # Resize the output shape to match the expected shape
    output_data[0] = rt.resize(output_data[0], (1,))

AttributeError: module 'onnxruntime' has no attribute 'resize'

#### Check that predictions are equals

In [43]:
test["predicted"] = pred
test["onnx_predicted"] = pred_onnx
test

,Open,High,Low,Volume,Close,SMA_3,SMA_5,SMA_15,SMA_30,SMA_50,...,close_3,close_4,close_5,high_1,high_2,high_3,high_4,high_5,predicted,onnx_predicted
Time,,,,,,,,,,,,,,,,,,,,,
2022-01-04 00:45:00,46276.98,46324.27,46257.91,20.01309,46311.71,46298.213333,46310.666,46370.960000,46344.401333,46378.6868,...,46324.28,46334.41,46391.94,46305.97,46324.28,46351.65,46410.42,46398.17,1,1
2022-01-04 00:46:00,46311.71,46334.87,46280.76,11.92522,46298.51,46295.730000,46303.486,46362.328667,46343.249000,46375.9980,...,46305.96,46324.28,46334.41,46324.27,46305.97,46324.28,46351.65,46410.42,1,1
2022-01-04 00:47:00,46298.50,46312.61,46291.95,5.98359,46304.50,46304.906667,46299.530,46355.416667,46343.678000,46373.6678,...,46276.97,46305.96,46324.28,46334.87,46324.27,46305.97,46324.28,46351.65,1,1
2022-01-04 00:48:00,46304.51,46324.00,46304.49,6.22721,46323.99,46309.000000,46303.136,46350.518667,46345.447333,46371.6740,...,46311.71,46276.97,46305.96,46312.61,46334.87,46324.27,46305.97,46324.28,0,0
2022-01-04 00:49:00,46324.00,46324.00,46290.00,15.71796,46290.00,46306.163333,46305.742,46340.122000,46345.714000,46368.5520,...,46298.51,46311.71,46276.97,46324.00,46312.61,46334.87,46324.27,46305.97,1,1
2022-01-04 00:50:00,46290.01,46296.53,46260.42,11.74081,46270.00,46294.663333,46297.400,46329.795333,46344.720333,46363.7380,...,46304.50,46298.51,46311.71,46324.00,46324.00,46312.61,46334.87,46324.27,1,1
2022-01-04 00:51:00,46265.44,46310.00,46261.60,15.76963,46309.99,46289.996667,46299.696,46324.315333,46346.208333,46360.3534,...,46323.99,46304.50,46298.51,46296.53,46324.00,46324.00,46312.61,46334.87,0,0
2022-01-04 00:52:00,46310.00,46310.00,46258.77,10.34325,46280.99,46286.993333,46294.994,46317.311333,46346.497000,46355.8458,...,46290.00,46323.99,46304.50,46310.00,46296.53,46324.00,46324.00,46312.61,1,1
2022-01-04 00:53:00,46281.00,46290.10,46240.56,16.15161,46240.57,46277.183333,46278.310,46307.677333,46346.338667,46351.9396,...,46270.00,46290.00,46323.99,46310.00,46310.00,46296.53,46324.00,46324.00,1,1


In [ ]:
session = rt.InferenceSession('model.onnx', sess_options={'shape_inference_mode': 'ONNX_USE_STATIC_SHAPE'})
outputs = session.run(inputs={'input_tensor': input_tensor},
                      outputs=['output_tensor'])
input_name = session.get_inputs()[0].name

session = onnxruntime.InferenceSession(model_path, sess_options={'shape_inference_mode': 'ONNX_USE_STATIC_SHAPE'}, providers=providers)
outputs = session.run(input_feed, output_feed, enable_profiling=enable_profiling, execution_mode=execution_mode)


pred_onnx = session.run({input_name: test.values.astype(np.float32)}, outputs=['output_tensor'])[0]
reshaped_tensor = reshape(outputs['output_tensor'], (10,))